# Imputing the data for model pipeline, using linear regression on numerical features and logistic regression on binary features

In [1]:
# Add 'graph' environment to PATH
import sys
sys.path.append('/home/ski/anaconda3/envs/graph/lib/python3.8/site-packages')

# User defined functions
import base_methods
from importlib import reload
from base_methods import load_the_csvs, reformat_large_tick_values, column_formatter

# Import data analysis packages
import numpy as np
import pandas as pd
# import csv
import warnings
# import os
# import time
# import math
import re
warnings.filterwarnings('ignore')

# Import plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)
sns.set_style("white")

# ML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn import linear_model

## Loading data

In [2]:
# Store path to notebook
PWD = !pwd
PWD = PWD[0]

# Set global paths to data folders
!mkdir {PWD}/files/output/
print()
INPUT = PWD + '/files/csv/'
OUTPUT = PWD + '/files/output/'

# Load
df = load_the_csvs(loc=OUTPUT, data=['baseline'], verbose=True)

mkdir: cannot create directory ‘/home/ski/Desktop/crunchbase-p1-machine-learning/files/output/’: File exists

/HOME/SKI/DESKTOP/CRUNCHBASE-P1-MACHINE-LEARNING/FILES/OUTPUT/BASELINE.CSV
BASELINE shape: (1131325, 61)
BASELINE columns: ['uuid', 'p1_tag', 'rank', 'employee_count', 'country_code', 'category_groups_list', 'total_funding_usd', 'age', 'ind_1', 'ind_2', 'ind_3', 'ind_4', 'ind_5', 'ind_6', 'ind_7', 'ind_8', 'ind_9', 'ind_10', 'ind_11', 'ind_12', 'ind_13', 'ind_14', 'ind_15', 'ind_16', 'ind_17', 'ind_18', 'ind_19', 'ind_20', 'ind_21', 'ind_22', 'ind_23', 'ind_24', 'ind_25', 'ind_26', 'ind_27', 'ind_28', 'ind_29', 'ind_30', 'ind_31', 'ind_32', 'ind_33', 'ind_34', 'ind_35', 'ind_36', 'ind_37', 'ind_38', 'ind_39', 'ind_40', 'ind_41', 'ind_42', 'ind_43', 'ind_44', 'ind_45', 'ind_46', 'cont_AF', 'cont_AS', 'cont_EU', 'cont_NA', 'cont_OC', 'cont_SA', 'continent_code']



In [11]:
df.head(3)

,uuid,p1_tag,rank,employee_count,country_code,category_groups_list,total_funding_usd,age,ind_1,ind_2,...,ind_44,ind_45,ind_46,cont_AF,cont_AS,cont_EU,cont_NA,cont_OC,cont_SA,continent_code
0,e1393508-30ea-8a36-3f96-dd3226033abd,0,102736.0,3.0,USA,"Content and Publishing,Internet Services,Media...",39750000.0,15.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,1,0,0,NorAm
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,0,24985.0,7.0,USA,"Information Technology,Internet Services,Priva...",NaN,24.0,1.0,1.0,...,0.0,0.0,0.0,0,0,0,1,0,0,NorAm
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,0,9319.0,3.0,USA,"Internet Services,Media and Entertainment",49000000.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,1,0,0,NorAm


## Check which variables have missing values and how many of the values are missing

In [8]:
num_missing = (df.isna().sum()).to_string()
print('NUMBER OF MISSING VALUES:\n\n',num_missing)

NUMBER OF MISSING VALUES:

 uuid                         0
p1_tag                       0
rank                     46704
employee_count          311994
country_code            192790
category_groups_list     98702
total_funding_usd       998411
age                     252682
ind_1                    98702
ind_2                    98702
ind_3                    98702
ind_4                    98702
ind_5                    98702
ind_6                    98702
ind_7                    98702
ind_8                    98702
ind_9                    98702
ind_10                   98702
ind_11                   98702
ind_12                   98702
ind_13                   98702
ind_14                   98702
ind_15                   98702
ind_16                   98702
ind_17                   98702
ind_18                   98702
ind_19                   98702
ind_20                   98702
ind_21                   98702
ind_22                   98702
ind_23                   98702
ind_24     

To check for any obviously unusual values, we look at a simple distribution for each variable:

In [5]:
df.describe()

,p1_tag,rank,employee_count,total_funding_usd,age,ind_1,ind_2,ind_3,ind_4,ind_5,...,ind_43,ind_44,ind_45,ind_46,cont_AF,cont_AS,cont_EU,cont_NA,cont_OC,cont_SA
count,1.131325e+06,1.084621e+06,819331.0,1.329140e+05,878643.0,1032623.0,1032623.0,1032623.0,1.032623e+06,1032623.0,...,1.032623e+06,1.032623e+06,1.032623e+06,1.032623e+06,1.131325e+06,1.131325e+06,1.131325e+06,1.131325e+06,1.131325e+06,1.131325e+06
mean,6.922856e-03,5.032787e+05,NaN,2.871436e+07,NaN,NaN,NaN,NaN,5.661011e-02,NaN,...,2.389526e-02,6.252289e-03,1.005554e-02,2.029419e-02,9.426115e-03,1.494142e-01,2.604504e-01,3.738501e-01,1.824233e-02,1.562416e-02
std,8.291523e-02,2.950290e+05,0.0,3.944204e+08,NaN,0.0,0.0,0.0,2.310791e-01,0.0,...,1.525879e-01,7.885742e-02,9.985352e-02,1.409912e-01,9.662956e-02,3.564964e-01,4.388806e-01,4.838248e-01,1.338266e-01,1.240164e-01
min,0.000000e+00,1.000000e+00,1.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.486870e+05,1.0,3.475760e+05,7.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,4.955420e+05,2.0,2.000000e+06,12.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,7.580730e+05,2.0,1.000000e+07,23.0,1.0,0.0,0.0,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.083386e+06,9.0,1.000000e+11,342.0,1.0,1.0,1.0,1.000000e+00,1.0,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


## Impute Total Funding Amount, Rank, Employee Count, and Age Using Linear Regression

In [93]:
#First impute all variables with a simple method (e.g. median), and then follow with regression imputation iteratively
imputer = SimpleImputer(missing_values=np.NaN, strategy='median')

variables = ['employee_count', 'total_funding_usd','rank', 'age']

for i in variables:
    df['imp_' + i] = imputer.fit_transform(df[i].values.reshape(-1,1))

#Remove ind columns for now
df_subset = df[['uuid','p1_tag','rank','age', 'total_funding_usd','employee_count','continent_code',
                'imp_employee_count', 'imp_total_funding_usd', 'imp_rank', 'imp_age']]

In [98]:
lin_reg_data = pd.DataFrame(columns = ['lin_' + i for i in variables])

for i in variables:
    lin_reg_data['lin_' + i] = df_subset['imp_' + i]
    parameters = list(set(df_subset.columns) - set(variables) - {'p1_tag'} - {'continent_code'} - {'uuid'} - {'imp_' + i})
    
    #Run linear regression to impute values
    model = linear_model.LinearRegression()
    model.fit(X =df_subset[parameters], y=df_subset['imp_'+i])
    
    #Save imputed values
    lin_reg_data.loc[df_subset[i].isna(), 'lin_' + i] = model.predict(df_subset[parameters])[df_subset[i].isna()]

In [101]:
#Add linear regression-imputed total_funding_usd and rank to original baseline dataset
df = pd.merge(df,lin_reg_data,how="inner",left_index=True,right_index=True)
df.drop(['imp_total_funding_usd','total_funding_usd','rank','imp_rank',
         'imp_employee_count','employee_count','age','imp_age'], inplace=True, axis=1)
df = df.rename(columns={'lin_total_funding_usd':'total_funding_usd','lin_rank':'rank', 
                        'lin_employee_count':'employee_count', 'lin_age':'age'})

#Check to make sure the right number of rows remain after merge
print('Rows in dataset = ',len(df))

In [104]:
#Save to CSV
df.to_csv(OUTPUT+'baseline_impute_linear.csv', index=False, sep=';')
print(f'OUTPUT TO CSV {OUTPUT}')

OUTPUT TO CSV /home/ski/Desktop/crunchbase-p1-machine-learning/files/output/


## Use Logistic Regression to Impute Industry and Continent Binary Variables

In [120]:
# First impute all variables with a simple method (in this case, just set to 0), and then follow with regression imputation iteratively
ind = ['ind_1', 'ind_2', 'ind_3', 'ind_4', 'ind_5', 'ind_6', 'ind_7', 'ind_8', 'ind_9', 'ind_10', 'ind_11', 'ind_12', 'ind_13',
       'ind_14', 'ind_15', 'ind_16', 'ind_17', 'ind_18', 'ind_19', 'ind_20', 'ind_21', 'ind_22', 'ind_23', 'ind_24', 'ind_25',
       'ind_26', 'ind_27', 'ind_28', 'ind_29', 'ind_30', 'ind_31', 'ind_32', 'ind_33', 'ind_34', 'ind_35', 'ind_36', 'ind_37',
       'ind_38', 'ind_39', 'ind_40', 'ind_41', 'ind_42', 'ind_43', 'ind_44', 'ind_45', 'ind_46']

cont = ['cont_AF', 'cont_AS', 'cont_EU', 'cont_NA', 'cont_OC', 'cont_SA']
ind = ind+cont

for i in ind:
    df['imp_' + i] = df[i].fillna(0)

In [123]:
#Subset to relevant variables for regression
df_2_subset = df.drop(['category_groups_list','country_code','continent_code','p1_tag'], axis = 1)
num_columns = ['rank','employee_count','age','total_funding_usd']

log_reg_data = pd.DataFrame(columns = ['log_' + i for i in ind])

for i in ind:
    print(i, sep=' ')
    log_reg_data['log_' + i] = df_2_subset['imp_' + i]
    parameters = list(set(df_2_subset.columns) - set(ind) - {'uuid'} - {'imp_' + i})
    
    #Run logisitic regression to impute values
    model = linear_model.LogisticRegression()
    model.fit(X = df_2_subset[parameters], y = df_2_subset['imp_' + i])
    
    #Save imputed values
    log_reg_data.loc[df_2_subset[i].isna(), 'log_' + i] = model.predict(df_2_subset[parameters])[df_2_subset[i].isna()]

ind_1
ind_2
ind_3
ind_4
ind_5
ind_6
ind_7
ind_8
ind_9
ind_10
ind_11
ind_12
ind_13
ind_14
ind_15
ind_16
ind_17
ind_18
ind_19
ind_20
ind_21
ind_22
ind_23
ind_24
ind_25
ind_26
ind_27
ind_28
ind_29
ind_30
ind_31
ind_32
ind_33
ind_34
ind_35
ind_36
ind_37
ind_38
ind_39
ind_40
ind_41
ind_42
ind_43
ind_44
ind_45
ind_46
cont_AF
cont_AS
cont_EU
cont_NA
cont_OC
cont_SA


In [126]:
#Add logistic regression-imputed variables to original baseline dataset
df = pd.merge(df, log_reg_data, how="inner",left_index=True,right_index=True)

#Drop original industry columns and columns with basic imputation (set na to 0)
imp = ['imp_ind_1', 'imp_ind_2', 'imp_ind_3', 'imp_ind_4', 'imp_ind_5', 'imp_ind_6', 'imp_ind_7', 'imp_ind_8', 'imp_ind_9',
       'imp_ind_10', 'imp_ind_11', 'imp_ind_12', 'imp_ind_13', 'imp_ind_14', 'imp_ind_15', 'imp_ind_16', 'imp_ind_17', 'imp_ind_18',
       'imp_ind_19', 'imp_ind_20', 'imp_ind_21', 'imp_ind_22', 'imp_ind_23', 'imp_ind_24', 'imp_ind_25', 'imp_ind_26', 'imp_ind_27',
       'imp_ind_28', 'imp_ind_29', 'imp_ind_30', 'imp_ind_31', 'imp_ind_32', 'imp_ind_33', 'imp_ind_34', 'imp_ind_35', 'imp_ind_36',
       'imp_ind_37', 'imp_ind_38', 'imp_ind_39', 'imp_ind_40', 'imp_ind_41', 'imp_ind_42', 'imp_ind_43', 'imp_ind_44', 'imp_ind_45',
       'imp_ind_46', 'imp_cont_AF', 'imp_cont_AS', 'imp_cont_EU', 'imp_cont_NA', 'imp_cont_OC', 'imp_cont_SA']

df.drop(imp, inplace=True, axis=1)
df.drop(ind, inplace=True, axis=1)

def log_rename(col_name):
    if re.match(r"^log_", col_name):
        return (col_name[4:])
    else:
        return col_name

,uuid,p1_tag,country_code,category_groups_list,continent_code,employee_count,total_funding_usd,rank,age,ind_1,...,ind_43,ind_44,ind_45,ind_46,cont_AF,cont_AS,cont_EU,cont_NA,cont_OC,cont_SA
0,e1393508-30ea-8a36-3f96-dd3226033abd,0,USA,"Content and Publishing,Internet Services,Media...",NorAm,3.000000,3.975000e+07,102736.00000,15.00000,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,0,USA,"Information Technology,Internet Services,Priva...",NorAm,7.000000,3.654675e+07,24985.00000,24.00000,1.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,0,USA,"Internet Services,Media and Entertainment",NorAm,3.000000,4.900000e+07,9319.00000,16.00000,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,0,USA,"Financial Services,Lending and Investments,Sof...",NorAm,4.000000,2.104107e+07,29637.00000,17.00000,1.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
4,df662812-7f97-0b43-9d3e-12f64f504fbb,0,USA,"Apps,Commerce and Shopping,Content and Publish...",NorAm,9.000000,2.335700e+09,222.00000,17.00000,1.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131320,81cc6b99-7b5d-4924-a802-3b934fb32009,0,BGD,"Commerce and Shopping,Food and Beverage,Transp...",AS,2.000000,6.117881e+06,503342.40625,0.00000,0.0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
1131321,f5a84fc5-6693-4cab-afc9-1abd7ea90b0f,0,MMR,"Content and Publishing,Media and Entertainment...",AS,2.000000,5.538228e+06,503300.43750,6.00000,0.0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
1131322,6395a6bc-f283-4e93-a76c-d693b0bc2b88,0,NaN,NaN,NaN,2.015625,4.958575e+06,503258.46875,17.59375,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1131323,4026b9ca-bdfb-405a-8b6b-7a5876f8618f,0,NaN,NaN,NaN,2.015625,4.958575e+06,503258.46875,17.59375,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [129]:
df.rename(columns=log_rename, inplace=True)
#Check to make sure the right number of rows remain after merge
print('Rows in dataset = ',len(df))
df.head()

Rows in dataset =  1131325


,uuid,p1_tag,country_code,category_groups_list,continent_code,employee_count,total_funding_usd,rank,age,ind_1,...,ind_43,ind_44,ind_45,ind_46,cont_AF,cont_AS,cont_EU,cont_NA,cont_OC,cont_SA
0,e1393508-30ea-8a36-3f96-dd3226033abd,0,USA,"Content and Publishing,Internet Services,Media...",NorAm,3.0,3.975000e+07,102736.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,0,USA,"Information Technology,Internet Services,Priva...",NorAm,7.0,3.654675e+07,24985.0,24.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,0,USA,"Internet Services,Media and Entertainment",NorAm,3.0,4.900000e+07,9319.0,16.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,0,USA,"Financial Services,Lending and Investments,Sof...",NorAm,4.0,2.104107e+07,29637.0,17.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
4,df662812-7f97-0b43-9d3e-12f64f504fbb,0,USA,"Apps,Commerce and Shopping,Content and Publish...",NorAm,9.0,2.335700e+09,222.0,17.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0


In [130]:
#Check number of missing values
num_missing = (df.isna().sum()).to_string()
print('NUMBER OF MISSING VALUES:\n\n',num_missing)

NUMBER OF MISSING VALUES:

 uuid                         0
p1_tag                       0
country_code            192790
category_groups_list     98702
continent_code          195711
employee_count               0
total_funding_usd            0
rank                         0
age                          0
ind_1                        0
ind_2                        0
ind_3                        0
ind_4                        0
ind_5                        0
ind_6                        0
ind_7                        0
ind_8                        0
ind_9                        0
ind_10                       0
ind_11                       0
ind_12                       0
ind_13                       0
ind_14                       0
ind_15                       0
ind_16                       0
ind_17                       0
ind_18                       0
ind_19                       0
ind_20                       0
ind_21                       0
ind_22                       0
ind_23     

In [131]:
#Save to CSV
df.to_csv(OUTPUT+'baseline_impute_complete.csv', index=False, sep=';')
print(f'OUTPUT TO CSV {OUTPUT}')

OUTPUT TO CSV /home/ski/Desktop/crunchbase-p1-machine-learning/files/output/
